In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pandavro as pdx
from shl.prepare import fill_na, calculate_abs_values, calculate_change, calculate_pct_change, calculate_window, calculate_shift
from typing import List, Optional, Tuple

os.chdir('../')
data_path = './data'

pd.set_option('display.max_columns', None)

In [23]:
def prepare_features(df, df_label, abs_column_patterns, window_sizes, window_center, window_functions, shift_periods, shift_column_patterns):  
        
    # Add time rounded to seconds
    df['epoch_time'] = df['epoch_time'].round(-3)
    
    # Group values by rounded time
    df = df.groupby(['epoch_time'], as_index=False).mean().drop(['Ignore1', 'Ignore2'], axis=1)
    
    # Join with label
    if df_label is None:
        df_label = df[['epoch_time']]  
    
    df = df_label.merge(df, how='left').drop('label', axis=1, errors='ignore')       
    
    # Ensure that values are sorted before calculating diff
    df = df.sort_values(by='epoch_time')    
    
    # Calculate difference to get distance and speed
    df_diff = calculate_change(df)
    
    # Additional features
    df_diff['distance'] = (df_diff['Latitude_change'].pow(2) + df_diff['Longitude_change'].pow(2)).pow(0.5)
    df_diff['speed'] = df_diff['distance']/df_diff['epoch_time_change']
    df_diff['vertical_speed'] = df_diff['Altitude_change']/df_diff['epoch_time_change']
    df_diff['direction'] = df_diff['Longitude_change']/df_diff['Latitude_change']
    df_diff['vertical_direction'] = df_diff['Altitude_change']/df_diff['distance']
    df_diff['is_stop'] = df_diff['speed'].apply(lambda x: 1 if x == 0 else 0)
    
    df_diff.drop(['epoch_time_change', 'accuracy_change', 'Latitude_change', 'Longitude_change', 'Altitude_change', 'distance'], axis=1, inplace=True) 
    fill_na(df_diff)
    
    # Second diff to get change of the new features
    df_diff_2 = calculate_change(df_diff[['speed', 'vertical_speed', 'direction', 'vertical_direction']])               
    
    # Percent change for accuracy
    df_pct = calculate_pct_change(df[['accuracy']])
    
    # Merge new features back to the main dataframe
    df = df[['epoch_time']].merge(df_diff, left_index=True, right_index=True, how='left') \
                           .merge(df_diff_2, left_index=True, right_index=True, how='left') \
                           .merge(df_pct, left_index=True, right_index=True, how='left')
    
    # Additional abs values features
    abs_columns = [col for col in df.columns for pattern in abs_column_patterns if pattern in col]
    calculate_abs_values(df, columns=abs_columns)
          
    # Add moving windows features
    df = calculate_window(df, columns=None, functions=window_functions, window_sizes=window_sizes, window_center=True)       
    
    # Add shifted features
    calculate_shift(df, periods=shift_periods, columns_patterns=shift_column_patterns) 
               
    return df

In [12]:
def split_files_into_parts(df, column_pattern, column_pattern_exclude):
    columns = df.columns
    if column_pattern is not None:
        columns = ['epoch_time'] + [col for col in columns if column_pattern in col]    
    if column_pattern_exclude is not None:
        columns_exclude = [col for col in columns for pattern_excl in column_pattern_exclude if pattern_excl in col]
        columns = [col for col in columns if col not in columns_exclude] 
    return df[columns]

In [24]:
# Run processing

settings = {
'abs_column_patterns': ['speed_change', 'direction_change'],
'window_sizes': [5, 10, 30, 60, 300],
'window_center': True,
'window_functions': ['mean', 'std'],
'shift_periods': [5, 10, 30, 60, 300],
'shift_column_patterns': ['window_5_', 'window_10_', 'window_30_', 'window_60_', 'window_300_'],
}

input_file_name = 'Location.parquet'
# input_file_name_label = 'Label.parquet'
data_types = ['validate', 'train', 'test']
full_input_file_names = [os.path.join(data_path, data_type, input_file_name) for data_type in data_types]
full_input_file_names_label = [os.path.join(data_path, data_type, input_file_name_label) for data_type in data_types]

base_output_file_name = 'features_location'
output_format = 'parquet'
output_file_name = base_output_file_name + '.' + output_format
full_output_file_names = [os.path.join(data_path, data_type, output_file_name) for data_type in data_types]

for file_in, file_in_label, file_out in zip(full_input_file_names, full_input_file_names_label, full_output_file_names):
    df = pd.read_parquet(file_in)
    df_label = None
    if os.path.exists(file_in_label):
        df_label = pd.read_parquet(file_in_label)
    df = prepare_features(df, df_label, **settings)
    if output_format == 'parquet':
        df.to_parquet(file_out, index=False)
    elif output_format == 'avro':
        pdx.to_avro(file_out, df)        


In [26]:
df

,epoch_time,speed,vertical_speed,direction,vertical_direction,is_stop,speed_change,vertical_speed_change,direction_change,vertical_direction_change,accuracy_pct_change,abs_speed_change,abs_vertical_speed_change,abs_direction_change,abs_vertical_direction_change,speed_window_5_mean,speed_window_5_std,vertical_speed_window_5_mean,vertical_speed_window_5_std,direction_window_5_mean,direction_window_5_std,vertical_direction_window_5_mean,vertical_direction_window_5_std,is_stop_window_5_mean,is_stop_window_5_std,speed_change_window_5_mean,speed_change_window_5_std,vertical_speed_change_window_5_mean,vertical_speed_change_window_5_std,direction_change_window_5_mean,direction_change_window_5_std,vertical_direction_change_window_5_mean,vertical_direction_change_window_5_std,accuracy_pct_change_window_5_mean,accuracy_pct_change_window_5_std,abs_speed_change_window_5_mean,abs_speed_change_window_5_std,abs_vertical_speed_change_window_5_mean,abs_vertical_speed_change_window_5_std,abs_direction_change_window_5_mean,abs_direction_change_window_5_std,abs_vertical_direction_change_window_5_mean,abs_vertical_direction_change_window_5_std,speed_window_10_mean,speed_window_10_std,vertical_speed_window_10_mean,vertical_speed_window_10_std,direction_window_10_mean,direction_window_10_std,vertical_direction_window_10_mean,vertical_direction_window_10_std,is_stop_window_10_mean,is_stop_window_10_std,speed_change_window_10_mean,speed_change_window_10_std,vertical_speed_change_window_10_mean,vertical_speed_change_window_10_std,direction_change_window_10_mean,direction_change_window_10_std,vertical_direction_change_window_10_mean,vertical_direction_change_window_10_std,accuracy_pct_change_window_10_mean,accuracy_pct_change_window_10_std,abs_speed_change_window_10_mean,abs_speed_change_window_10_std,abs_vertical_speed_change_window_10_mean,abs_vertical_speed_change_window_10_std,abs_direction_change_window_10_mean,abs_direction_change_window_10_std,abs_vertical_direction_change_window_10_mean,abs_vertical_direction_change_window_10_std,speed_window_30_mean,speed_window_30_std,vertical_speed_window_30_mean,vertical_speed_window_30_std,direction_window_30_mean,direction_window_30_std,vertical_direction_window_30_mean,vertical_direction_window_30_std,is_stop_window_30_mean,is_stop_window_30_std,speed_change_window_30_mean,speed_change_window_30_std,vertical_speed_change_window_30_mean,vertical_speed_change_window_30_std,direction_change_window_30_mean,direction_change_window_30_std,vertical_direction_change_window_30_mean,vertical_direction_change_window_30_std,accuracy_pct_change_window_30_mean,accuracy_pct_change_window_30_std,abs_speed_change_window_30_mean,abs_speed_change_window_30_std,abs_vertical_speed_change_window_30_mean,abs_vertical_speed_change_window_30_std,abs_direction_change_window_30_mean,abs_direction_change_window_30_std,abs_vertical_direction_change_window_30_mean,abs_vertical_direction_change_window_30_std,speed_window_60_mean,speed_window_60_std,vertical_speed_window_60_mean,vertical_speed_window_60_std,direction_window_60_mean,direction_window_60_std,vertical_direction_window_60_mean,vertical_direction_window_60_std,is_stop_window_60_mean,is_stop_window_60_std,speed_change_window_60_mean,speed_change_window_60_std,vertical_speed_change_window_60_mean,vertical_speed_change_window_60_std,direction_change_window_60_mean,direction_change_window_60_std,vertical_direction_change_window_60_mean,vertical_direction_change_window_60_std,accuracy_pct_change_window_60_mean,accuracy_pct_change_window_60_std,abs_speed_change_window_60_mean,abs_speed_change_window_60_std,abs_vertical_speed_change_window_60_mean,abs_vertical_speed_change_window_60_std,abs_direction_change_window_60_mean,abs_direction_change_window_60_std,abs_vertical_direction_change_window_60_mean,abs_vertical_direction_change_window_60_std,speed_window_300_mean,speed_window_300_std,vertical_speed_window_300_mean,vertical_speed_window_300_std,direction_window_300_mean,direction_wi

In [25]:
df.sum().sum()

3.2709968809179686e+17

In [22]:
df.isnull().sum().sum()

0

In [17]:
# Split files into parts to avoid error when loading from Google Storage to BigQuery

"""
Split is based on logical parts of the data:
0 - base features
1 - window values
2 - shifted values (future)
3 - shifted values (past)

Use file_parts, column_patterns and column_patterns_exclude to configure this.
"""

file_parts = np.arange(4)
column_patterns = [None, 'window', 'shift', 'shift']
column_patterns_exclude = [['window', 'shift'], ['shift'], ['past'], ['future']]

input_file_name = 'features_location.parquet'
data_types = ['validate', 'train', 'test']
full_input_file_names = [os.path.join(data_path, data_type, input_file_name) for data_type in data_types]

output_format = 'parquet'
base_output_file_name = input_file_name.split('.')[0] + '_part_'
base_output_file_names = [base_output_file_name + str(part) + '.' + output_format for part in file_parts]

for data_type, file_in in zip(data_types, full_input_file_names):
    df_input = pd.read_parquet(file_in)
    for part, col_pattern, col_pattern_excl in zip(file_parts, column_patterns, column_patterns_exclude):        
        df = split_files_into_parts(df_input, col_pattern, col_pattern_excl)
        output_file_name = os.path.join(data_path, data_type, base_output_file_name + str(part) + '.' + output_format)
        df.to_parquet(output_file_name, index=False)        


In [4]:
df

,epoch_time,speed,vertical_speed,direction,vertical_direction,speed_change,vertical_speed_change,direction_change,vertical_direction_change,accuracy_pct_change,abs_speed_change,abs_vertical_speed_change,abs_direction_change,abs_vertical_direction_change,speed_window_5_mean,speed_window_5_std,vertical_speed_window_5_mean,vertical_speed_window_5_std,direction_window_5_mean,direction_window_5_std,vertical_direction_window_5_mean,vertical_direction_window_5_std,speed_change_window_5_mean,speed_change_window_5_std,vertical_speed_change_window_5_mean,vertical_speed_change_window_5_std,direction_change_window_5_mean,direction_change_window_5_std,vertical_direction_change_window_5_mean,vertical_direction_change_window_5_std,accuracy_pct_change_window_5_mean,accuracy_pct_change_window_5_std,abs_speed_change_window_5_mean,abs_speed_change_window_5_std,abs_vertical_speed_change_window_5_mean,abs_vertical_speed_change_window_5_std,abs_direction_change_window_5_mean,abs_direction_change_window_5_std,abs_vertical_direction_change_window_5_mean,abs_vertical_direction_change_window_5_std,speed_window_10_mean,speed_window_10_std,vertical_speed_window_10_mean,vertical_speed_window_10_std,direction_window_10_mean,direction_window_10_std,vertical_direction_window_10_mean,vertical_direction_window_10_std,speed_change_window_10_mean,speed_change_window_10_std,vertical_speed_change_window_10_mean,vertical_speed_change_window_10_std,direction_change_window_10_mean,direction_change_window_10_std,vertical_direction_change_window_10_mean,vertical_direction_change_window_10_std,accuracy_pct_change_window_10_mean,accuracy_pct_change_window_10_std,abs_speed_change_window_10_mean,abs_speed_change_window_10_std,abs_vertical_speed_change_window_10_mean,abs_vertical_speed_change_window_10_std,abs_direction_change_window_10_mean,abs_direction_change_window_10_std,abs_vertical_direction_change_window_10_mean,abs_vertical_direction_change_window_10_std,speed_window_30_mean,speed_window_30_std,vertical_speed_window_30_mean,vertical_speed_window_30_std,direction_window_30_mean,direction_window_30_std,vertical_direction_window_30_mean,vertical_direction_window_30_std,speed_change_window_30_mean,speed_change_window_30_std,vertical_speed_change_window_30_mean,vertical_speed_change_window_30_std,direction_change_window_30_mean,direction_change_window_30_std,vertical_direction_change_window_30_mean,vertical_direction_change_window_30_std,accuracy_pct_change_window_30_mean,accuracy_pct_change_window_30_std,abs_speed_change_window_30_mean,abs_speed_change_window_30_std,abs_vertical_speed_change_window_30_mean,abs_vertical_speed_change_window_30_std,abs_direction_change_window_30_mean,abs_direction_change_window_30_std,abs_vertical_direction_change_window_30_mean,abs_vertical_direction_change_window_30_std,speed_window_60_mean,speed_window_60_std,vertical_speed_window_60_mean,vertical_speed_window_60_std,direction_window_60_mean,direction_window_60_std,vertical_direction_window_60_mean,vertical_direction_window_60_std,speed_change_window_60_mean,speed_change_window_60_std,vertical_speed_change_window_60_mean,vertical_speed_change_window_60_std,direction_change_window_60_mean,direction_change_window_60_std,vertical_direction_change_window_60_mean,vertical_direction_change_window_60_std,accuracy_pct_change_window_60_mean,accuracy_pct_change_window_60_std,abs_speed_change_window_60_mean,abs_speed_change_window_60_std,abs_vertical_speed_change_window_60_mean,abs_vertical_speed_change_window_60_std,abs_direction_change_window_60_mean,abs_direction_change_window_60_std,abs_vertical_direction_change_window_60_mean,abs_vertical_direction_change_window_60_std,speed_window_300_mean,speed_window_300_std,vertical_speed_window_300_mean,vertical_speed_window_300_std,direction_window_300_mean,direction_window_300_std,vertical_direction_window_300_mean,vertical_direction_window_300_std,speed_change_window_300_mean,speed_change_window_300_std,vertical_speed_change_window_300_mean,vertical_

### Export for analysis

In [20]:
# Join labels for analysis
# features_file = './data/validate/features_location_part_1.parquet'
# features = pd.read_parquet(features_file)
# features['epoch_time'] = features['epoch_time'].round(-3)

# label_file = './data/validate/Label.parquet'
# label = pd.read_parquet(label_file)

# features = label.merge(features).sort_values(by='epoch_time')
# features.to_csv('./data/location_analysis.csv')
# features

,epoch_time,label,speed_window_5_mean,speed_window_5_std,vertical_speed_window_5_mean,vertical_speed_window_5_std,direction_window_5_mean,direction_window_5_std,vertical_direction_window_5_mean,vertical_direction_window_5_std,is_stop_window_5_mean,is_stop_window_5_std,speed_change_window_5_mean,speed_change_window_5_std,vertical_speed_change_window_5_mean,vertical_speed_change_window_5_std,direction_change_window_5_mean,direction_change_window_5_std,vertical_direction_change_window_5_mean,vertical_direction_change_window_5_std,accuracy_pct_change_window_5_mean,accuracy_pct_change_window_5_std,abs_speed_change_window_5_mean,abs_speed_change_window_5_std,abs_vertical_speed_change_window_5_mean,abs_vertical_speed_change_window_5_std,abs_direction_change_window_5_mean,abs_direction_change_window_5_std,abs_vertical_direction_change_window_5_mean,abs_vertical_direction_change_window_5_std,speed_window_10_mean,speed_window_10_std,vertical_speed_window_10_mean,vertical_speed_window_10_std,direction_window_10_mean,direction_window_10_std,vertical_direction_window_10_mean,vertical_direction_window_10_std,is_stop_window_10_mean,is_stop_window_10_std,speed_change_window_10_mean,speed_change_window_10_std,vertical_speed_change_window_10_mean,vertical_speed_change_window_10_std,direction_change_window_10_mean,direction_change_window_10_std,vertical_direction_change_window_10_mean,vertical_direction_change_window_10_std,accuracy_pct_change_window_10_mean,accuracy_pct_change_window_10_std,abs_speed_change_window_10_mean,abs_speed_change_window_10_std,abs_vertical_speed_change_window_10_mean,abs_vertical_speed_change_window_10_std,abs_direction_change_window_10_mean,abs_direction_change_window_10_std,abs_vertical_direction_change_window_10_mean,abs_vertical_direction_change_window_10_std,speed_window_30_mean,speed_window_30_std,vertical_speed_window_30_mean,vertical_speed_window_30_std,direction_window_30_mean,direction_window_30_std,vertical_direction_window_30_mean,vertical_direction_window_30_std,is_stop_window_30_mean,is_stop_window_30_std,speed_change_window_30_mean,speed_change_window_30_std,vertical_speed_change_window_30_mean,vertical_speed_change_window_30_std,direction_change_window_30_mean,direction_change_window_30_std,vertical_direction_change_window_30_mean,vertical_direction_change_window_30_std,accuracy_pct_change_window_30_mean,accuracy_pct_change_window_30_std,abs_speed_change_window_30_mean,abs_speed_change_window_30_std,abs_vertical_speed_change_window_30_mean,abs_vertical_speed_change_window_30_std,abs_direction_change_window_30_mean,abs_direction_change_window_30_std,abs_vertical_direction_change_window_30_mean,abs_vertical_direction_change_window_30_std,speed_window_60_mean,speed_window_60_std,vertical_speed_window_60_mean,vertical_speed_window_60_std,direction_window_60_mean,direction_window_60_std,vertical_direction_window_60_mean,vertical_direction_window_60_std,is_stop_window_60_mean,is_stop_window_60_std,speed_change_window_60_mean,speed_change_window_60_std,vertical_speed_change_window_60_mean,vertical_speed_change_window_60_std,direction_change_window_60_mean,direction_change_window_60_std,vertical_direction_change_window_60_mean,vertical_direction_change_window_60_std,accuracy_pct_change_window_60_mean,accuracy_pct_change_window_60_std,abs_speed_change_window_60_mean,abs_speed_change_window_60_std,abs_vertical_speed_change_window_60_mean,abs_vertical_speed_change_window_60_std,abs_direction_change_window_60_mean,abs_direction_change_window_60_std,abs_vertical_direction_change_window_60_mean,abs_vertical_direction_change_window_60_std,speed_window_300_mean,speed_window_300_std,vertical_speed_window_300_mean,vertical_speed_window_300_std,direction_window_300_mean,direction_window_300_std,vertical_direction_window_300_mean,vertical_direction_window_300_std,is_stop_window_300_mean,is_stop_window_300_std,speed_change_window_300_mean,speed_change_window_300_std,vertical_speed_change_window_300_mean,vertical_speed_chang